In [48]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs


In [54]:
"""
TO DO:
- Entrar a la página de la facultad de ingenieria y probar la clave 1413 (Introducción a la Economia) y extraer la tabla de datos
- Colocar los datos en un excel y guardarlo
- Generalizar la función para cualquier clabe
- Anidar los datos extraidos en un único excell
"""


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#Accede al navegador y a la página de la facultad
driver = webdriver.Chrome()
driver.get("https://www.ssa.ingenieria.unam.mx/horarios.html")

campo_clave = driver.find_element("id","clave")
buscar = driver.find_element("id","buscar")
campo_clave.send_keys("1413")
buscar.click()

"""
#Localizacion de los inputs dentro de un frame en HTML
frame = driver.find_element("xpath","/html/frameset/frame")
driver.switch_to.frame(frame)
#Campo de entrada para el RFC
entradaRFC = driver.find_element("name", "ctl00$ContentPlaceHolder1$txtRFC")
entradaRFC.send_keys("DEGM950723")
time.sleep(tiempoCaptura)
#Campo de entrada para en numero de trabajador
entradaNumUser = driver.find_element("name", "ctl00$ContentPlaceHolder1$txtNumeroTrabajador")
entradaNumUser.send_keys("897047")
time.sleep(tiempoCaptura)
#intpass.send_keys(Keys.ENTER)
#Busqueda del boton login, y dar click
login = driver.find_element("name", "ctl00$ContentPlaceHolder1$btnEntrar")
login.click()
"""
time.sleep(5)

driver.close()




In [55]:
import re
import requests
from colorama import Fore

website = "https://www.ssa.ingenieria.unam.mx/horarios.html"
resultado = requests.get(website)
content = resultado.text

NameError: name 'website' is not defined

In [24]:
df = pd.read_excel('Excel/Economia.xlsx')
lista_horarios = list()
offset = 0
claves_excluidas = [1537]
open_link = False #bandera booleana para materias vinculadas (mismo grupo pero horario asimetrico)

In [25]:
for clave in claves_excluidas:
   df.drop(df[df['Clave']==clave].index, inplace=True)
materias = len(df.Clave.unique())
df.Clave.unique()
df


,Clave,Gpo,Profesor,Tipo,Horario,Días,Cupo,Nombre,Puntaje
0,5138,3,M.I. RAUL RUVALCABA MORALES,L+,09:00 a 11:00,Mie,26,L.Disp. Electr,9.6
1,1503,2,M.I. LUIS SERGIO DURAN ARENAS,T,11:00 a 13:00,"Lun, Mie",35,Est y Prog de Comp,8.6
2,5138,9,ING. JESUS MARIA FRANCISCO HERNANDEZ MORALES,L+,10:00 a 12:00,Vie,26,L.Disp. Electr,8.6
3,442,2,ING. IVAN VLADIMIR MEZA RUIZ,T,15:00 a 17:00,"Lun, Mie",35,Leng Form y Autom,8.5
4,1503,5,M.I. PEDRO IGNACIO RINCON GOMEZ,T,14:00 a 16:00,"Mar, Jue",35,Est y Prog de Comp,8.3
5,5138,6,ING. ROMAN VICTORIANO OSORIO COMPARAN,L+,11:30 a 13:30,Jue,26,L.Disp. Electr,8.3
6,138,3,DRA. ROSALBA MARTINEZ LOPEZ,T,16:00 a 18:00,"Lun, Mie",41,Disp. Electr,8.0
7,138,8,DRA. ROSALBA MARTINEZ LOPEZ,T,16:00 a 18:00,"Mar, Jue",41,Disp. Electr,8.0
8,1531,4,ING. MARCO ANTONIO DELGADO GONZALEZ,T,15:00 a 17:00,"Mar, Jue",35,Ing de Softw,8.0
9,1503,6,M.I. MIGUEL ISRAEL BARRAGAN OCAMPO,T,16:00 a 18:00,"Mar, Jue",35,Est y Prog de Comp,7.8


In [26]:
def noHaySolape(horario,check_df,n):    
    """
    Verifica si existe un solape entre un potencial horario y una materia (renglón del dataframe)

    Parameters
    ----------
        horario: dataFrame
            Es el horario que se va armando
        check_df: DataFrame
            Es el df con las materias
        n: int
            Es el renglón actual
    """
    global open_link
    check = check_df.iloc[n]
    
    for i in range(0,len(horario)): 
        v = horario.iloc[i,9:15] * check.iloc[9:15] #Checa el vector de días de la semana y los multiplica dia a dia, si vale cero, entonces no comparten días
        if ((not v.equals(cero)) and (haySolapeHoras(horario.iloc[i],check))): #Si comparten días y ademas horas entonces sí hay solape
            return False

    #Cuando haya salido del for, significa que no hay ningún posible solape
    if(open_link == True): #Si la materia que se está comprobando esta linkeada
        #if isOpen_link(horario,check):
        if n<(len(df)-1):
            next = df.iloc[n+1]
            if not isOpen_link(horario,next):#Verifica si la materia que se está checando está vinculada a alguna materia del horario
                open_link = False #ya se ha pagado el vinculo que se debía
        else:
            open_link = False
            
    return True

In [27]:
#Verificación por horario
def haySolapeHoras(a,b): #DataFrame, comparativa a comparativa b
    if b.Inicio_min - a.Inicio_min == 0: #empiezan a la misma hora
        return True
    if b.Inicio_min - a.Inicio_min > 0: #b empieza después que a
        if(b.Inicio_min - a.Inicio_min < a.Duracion*60):
            return True
    else:
        if(abs(b.Inicio_min - a.Inicio_min) < b.Duracion*60):
            return True 
    return False
    

# Get dummies de los días

In [28]:
#orden = list(['Lun','Mar','Mie','Jue','Vie','Sab'])
dias = pd.Series(df.Días).str.get_dummies(sep = ', ')
orden = list(['Lun','Mar','Mie','Jue','Vie','Sab'])
dias[orden] = 0
dias = dias[orden]
dias += pd.Series(df.Días).str.get_dummies(sep = ', ')
dias.fillna(0,inplace=True,downcast = 'int64')

In [29]:
df = pd.concat([df,dias],axis=1)

# Get dummies de las horas

In [30]:
horas = df.Horario.str.split(" a ")

In [31]:
horas = pd.DataFrame.from_dict(dict(zip(horas.index, horas.values))).transpose()
horas.columns = ['Inicio','Fin']

In [32]:
df = pd.concat([df,horas],axis=1)


In [33]:
hrs_y_min = df.Inicio.str.split(":")
hrs_y_min.str[0]
minutos = hrs_y_min.str[0].astype(int)*60 +  hrs_y_min.str[1].astype(int)
minutos
df = df.assign(Inicio_min = minutos)

hrs_y_min = df.Fin.str.split(":")
hrs_y_min.str[0]
minutos = hrs_y_min.str[0].astype(int)*60 +  hrs_y_min.str[1].astype(int)
minutos
df = df.assign(Fin_min = minutos)
df

duracion = df.Fin_min - df.Inicio_min
df = df.assign(Duracion = duracion/60)


In [34]:
def generarHorarios():
    """
    Genera los horarios, realiza un arbol de posibilidades por cada opcion de la primer materia
    """
    n=0
    global open_link
    while(df.iloc[n].Clave == df.Clave.unique()[0]):
        open_link = False
        horario = pd.DataFrame(columns = df.columns) #crea un df con las mismas columnas para ir armando el horario
        combinarMaterias(df,horario,n)
        n = n+1

In [35]:
def combinarMaterias(df,horario,renglon_actual):
    """
    Es una función recursiva que toma los parámetros para ir concatenando un horario

    Parameters
    ------------
        df: dataFrame
            El dataframe que contiene todas las materias
        horario: dataFrame
            El horario que se va formando
        renglon_actual: int
            El indice del renglón del df que se concatena
    
    """
    global open_link
    if renglon_actual < len(df):
        new_row = df.copy()[df.index == renglon_actual]
        horario = pd.concat([horario, new_row])         #Copia el renglon actual al horario
        for i in range(renglon_actual+1,len(df)):
            if hayVinculoAtras(df,i):
                if not includes(horario,df.iloc[i-1]): #Esta vinculado con una rama anterior
                    continue

            if isClaveRepetida(horario,df.iloc[i]): 
                if not isOpen_link(horario,df.iloc[i]): #Si la materia es la misma pero con grupo diferente, la omite
                    continue
                #else la materia esta linkeada
            else: #Se tiene una materia diferente
                if open_link == True:
                    continue
                
            #isOpen_link(horario,df.iloc[i])
            temp = open_link
            if noHaySolape(horario,df,i):
                combinarMaterias(df,horario,i) #Al ser recursiva, va concatenando las filas del excel que no se solapen y sean diferentes materias
                open_link = temp
                #isOpen_link(horario,df.iloc[i]) #Recupera el estado de la bander open_link para eliminar opciones
        
        if(len(horario.Clave.unique()) == materias and open_link == False): #Comprueba la variable horarios llenos para ver si tiene que cumplir con cubrir todas las materias del excel y no haya una incompleta
                lista_horarios.append(horario)
                plotHorario(horario,len(lista_horarios))
       

In [36]:
def includes(horario,materia):
    for i in range(0,len(horario)): #Checa con todas las materias del horario
        if (materia.Gpo == horario.iloc[i].Gpo and materia.Clave == horario.iloc[i].Clave):
            return True
    return False

In [37]:
def hayVinculoAtras(df,n):
    if(n>1):
        if (df.iloc[n].Gpo == df.iloc[n-1].Gpo and df.iloc[n].Clave == df.iloc[n-1].Clave):
            return True
    return False

In [38]:
def isClaveRepetida(horario,materia):
    for i in range(0,len(horario)): #Checa con todas las materias del horario
        if (materia.Clave == horario.iloc[i].Clave):
            return True
    return False

In [39]:
def isOpen_link(horario,materia):
    """
    Cambia la bandera open_link si la materia está vinculada con alguna del horario
    Una materia está vinculada con el horario si tiene misma clave y grupo.

    Parameters
    ---------
        horario: DataFrame
            Es el horario que se está armando
        materia: Series
            Es una materia con la que se está comprobando
    """
    global open_link
    for i in range(0,len(horario)): #Checa con todas las materias
        if (materia.Gpo == horario.iloc[i].Gpo and materia.Clave == horario.iloc[i].Clave):
            open_link = True #Se vinculan únicamente si coincide alguna materia en grupo y clave
            return True
    return False
    

In [40]:
def plotMateria(df,gnt):
    import random
    R = random.uniform(0.3,0.75)
    G = random.uniform(0, 0.1)
    B = random.uniform(0.6, 1)
    
    i = 0
    hora = (df.Inicio_min/60)
    duracion = df.Duracion
    
    for cat in df.iloc[9:15]: #Lo que hace es particionar el df a los días de la semana
        if cat == 1:
            gnt.broken_barh([(i,1)], (hora, duracion) ,facecolors = (R, G, B))
            nombre_materia = df.Nombre[:10]                                                     #Corta el texto para que quepa en el cuadro
            nombre_profe = df.Profesor.split(" ")
            nombre_pila = nombre_profe[1]
            nombre_pila = nombre_pila[:5]
            apellido = nombre_profe[-2]
            #nombre_pila = df.Profesor[:7]
            inicial = apellido [:1]
            nombre = nombre_materia + "\n" + str(df.Gpo) + " " + nombre_pila  + " " + inicial                                    
            gnt.text(i+0.1,hora+1.5,nombre,color="white",fontweight = "bold",fontsize = "small") #x,y,texto,color,grosor y tamaño
        i = i+1


In [41]:
def plotHorario(horario, numero):
    import matplotlib.pyplot as plt
    fig, gnt = plt.subplots()
    gnt.set_ylim(6,23 )
    gnt.set_xlim(0,6)
    gnt.set_xlabel('Día')
    gnt.set_ylabel('Horas')

    #horas = ['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22']
    dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado']
    plt.gca().invert_yaxis()
    gnt.set_yticks(np.arange(7, 23,1))
    gnt.set_xticks(np.arange(0,6,1))
    gnt.set_xticklabels(dias,ha="left")
    gnt.grid(True)

    for i in range(0,len(horario)):
        plotMateria(horario.iloc[i],gnt)

    plt.savefig("Horarios/opción" + str(numero) + ".png")
    plt.close()

In [42]:
def clearCarpeta():
    import os, shutil
    folder = 'Horarios/'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('No fue posible borrar la carpeta %s. Excepción: %s' % (file_path, e))
    

In [43]:
def imprimirHorarios():
    clearCarpeta()
    if len(lista_horarios) == 0:
        print("No se puede generar ningún horario con las materias ingresadas. POSIBLE SOLAPE")
        return 0
    n=1
    for horario in lista_horarios:
        plotHorario(horario,n)
        #print(horario.iloc[:,0:8].drop(['Tipo','Cupo'],axis=1))
        horario.drop(horario.tail(1).index,inplace=True)
        n+=1
    

In [44]:
horario = pd.DataFrame(columns = df.columns)
horario
cero = pd.Series([0,0,0,0,0,0], index = orden).astype(object)
generarHorarios()
print("Horarios generados: " + str(len(lista_horarios)))
imprimirHorarios()


Horarios generados: 2
